In [1]:
import scrapy
import random
import pymongo
import pandas as pd
from bs4 import BeautifulSoup
from pymongo import TEXT
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys


In [6]:
def browse(url, proxy = None):
    #     url = 'https://httpbin.org/ip'
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    if proxy is not None:
        chrome_options.add_argument('--proxy-server=%s' % proxy)
    chrome_options.add_argument('--no-sandbox')
    driver = webdriver.Chrome('/home/ubuntu/chromedriver',chrome_options=chrome_options)
    driver.get(url)
    driver.implicitly_wait(3) # seconds
    page_source = driver.page_source
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.close()
    return page_source, soup
    
class CrawlMKListing():
    MONGO_URI = "localhost:27017"
    MONGO_DB = "news"
    MONGO_COLLECTION = "malaysiakini_v1_test1"
    DOMAIN = "https://www.malaysiakini.com"
    NEWS_LISTING_DOMAIN = "https://www.malaysiakini.com/stories/covid19"
    PROXY_LIST = ["socks4://120.50.56.137:40553","socks4://121.122.50.157:4145", 
                  "socks4://1.9.167.36:60489","socks4://1.9.111.145:4145",
                  "socks4://45.117.228.153:4145","socks4://45.117.228.97:4145",
                  "socks4://103.220.6.254:4145"
                 ]
    START_URL = NEWS_LISTING_DOMAIN
    
    def __init__(self):
        pass
    
    def crawl(self):
        CURRENT_PAGE_URL = self.START_URL
        self.init_mongo()
        proxy = random.sample(self.PROXY_LIST,1)[0]
        print("Using proxy:", proxy)
        page_source, soup = browse(url = CURRENT_PAGE_URL, proxy = proxy)
        _check = self.check(soup)
        # if _check is not True, retry the whole crawl for this page
        if _check: 
            to_insert, next_page = self.parse(soup)
            to_insert_2 = self.prevent_duplicate(to_insert)
            if len(to_insert_2) > 0:
                self.coll.insert_many(to_insert_2)
        else:
            print("False check.")
            
    def init_mongo(self):
        client = pymongo.MongoClient(self.MONGO_URI)
        self.coll = client[self.MONGO_DB][self.MONGO_COLLECTION]

    def check(self, soup):
        try:
            x = soup.find("title").get_text()
        except:
            return False
        if x is None:
            return False
        if x.find("Access denied") >= 0:
            return False
        return True

    def parse(self, soup):
        DOMAIN = self.DOMAIN
        NEWS_LISTING_DOMAIN = self.NEWS_LISTING_DOMAIN
        
        x = soup.find("div", "news").find_all("a")[:-1]
        titles = [j.find("h3").getText() for j in x]
        urls = [j.get("href") for j in x]
        urls = [DOMAIN + url.replace(DOMAIN, "") for url in urls]
        df = pd.DataFrame(dict(title = titles, url = urls)) #.sample(30)
        if df.shape[0] == 0:
            raise Exception("shape 0")
        to_insert = df.to_dict(orient="records")
        next_page = NEWS_LISTING_DOMAIN + soup.find("div", "news").find_all("a")[-1].get("href")
        return to_insert, next_page
    
    def prevent_duplicate(self, to_insert):
        url_list = [j.get("url") for j in to_insert]
        exist_url_list = [j.get("url") for j in list(self.coll.find({"url": {"$in": url_list}}, {"url":1}))]
        to_insert_2 = [j for j in to_insert if j.get("url") not in exist_url_list]
        return to_insert_2

In [8]:
%%time
C = CrawlMKListing()
C.crawl()

Using proxy: socks4://120.50.56.137:40553


/home/ubuntu/anaconda3/envs/covid/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


[{'title': 'Experts urge smokers and tobacco firms to quit for Covid-19', 'url': 'https://www.malaysiakini.com/news/519173'}, {'title': 'MCO: Courts impose non-custodial sentences in wake of Covid-19 fears', 'url': 'https://www.malaysiakini.com/news/519171'}, {'title': 'Police increase 100pct deployment of officers monitoring borders, waters', 'url': 'https://www.malaysiakini.com/news/519163'}, {'title': 'Maintain good hygiene, Health DG tells pet owners', 'url': 'https://www.malaysiakini.com/news/519156'}, {'title': "'Be responsible, be honest' - Aida's message after losing five family members to Covid-19", 'url': 'https://www.malaysiakini.com/news/519146'}, {'title': 'Indonesia makes wearing of face masks compulsory when away from home', 'url': 'https://www.malaysiakini.com/news/519125'}, {'title': "Factbox: Who is Dominic Raab, Britain's standby leader?", 'url': 'https://www.malaysiakini.com/news/519119'}, {'title': "Accused of 'piracy', US denies diverting masks bound for Germany",

TypeError: documents must be a non-empty list

In [84]:
%%time
url = "https://www.malaysiakini.com/stories/covid19?page=0"
proxy = PROXY_LIST[2]
coll = init_mongo()
page_source, soup = browse(url, proxy)
_check = check(soup)
first_parse_list = parse(soup)
coll.insert_many(first_parse_list)

In [28]:
import numpy as np
np.random.([1,2,3])

array([[[0.653419  , 0.47827985, 0.65871204],
        [0.23944157, 0.97640932, 0.82097919]]])

In [39]:
import random
random.sample([j.get("href") for j in x], len(x))

['/news/519093',
 '/news/518995',
 '/news/519065',
 '/news/519021',
 '/news/519107',
 '/news/519038',
 '/news/518999',
 '/news/518989',
 '/news/519074',
 '/news/519042',
 '/news/519171',
 '/news/519013',
 '/news/518992',
 '/news/519173',
 '/announcement/519136',
 '/news/519101',
 '/news/519090',
 '/news/519163',
 '/news/519049',
 '/news/519118',
 '/news/518996',
 '/news/519125',
 '/news/519156',
 '/letters/519046',
 '/news/519006',
 '/letters/519086',
 '/news/519015',
 '/news/519119',
 '/news/519146',
 '/letters/519104']

In [ ]:
x = "36.89.10.51:34115"

# url = "https://www.thestar.com.my/tag/covid-19+watch"
url = "https://www.malaysiakini.com/stories/covid19"
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
driver = webdriver.Chrome('/home/ubuntu/chromedriver',chrome_options=chrome_options)
driver.get(url)

## Click load more
# python_button = driver.find_elements_by_xpath('//*[@id="loadMorestories"]')[0]
# python_button.click()

## parse
source = driver.page_source
soup = BeautifulSoup(driver.page_source, 'html.parser')

driver.close()


In [ ]:
x = "36.89.10.51:34115"
proxies = {
  'http': 'http://128.199.217.244:47503',
  'https': 'https://128.199.217.244:47503',
#   'https': 'http://10.10.1.10:1080',
}
PROXY = x #"128.199.217.244:47503"

In [ ]:
z=soup.find("div", "news")
news_list = z.find_all("a")
len(news_list)

In [ ]:
news_list[0].get("href")

In [ ]:
import requests
proxies = {
  'http': 'http://128.199.217.244:47503',
  'https': 'https://128.199.217.244:47503',
#   'https': 'http://10.10.1.10:1080',
}

x = requests.get('http://example.org', proxies=proxies)

In [ ]:
soup

In [ ]:
url = "https://www.thestar.com.my/tag/covid-19+watch"

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('/home/ubuntu/chromedriver',chrome_options=chrome_options)
driver.get(url)
# driver.implicitly_wait(3) # seconds
python_button = driver.find_elements_by_xpath('//*[@id="loadMorestories"]')[0]
python_button.click()
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.close()
# # click radio button

In [ ]:
# soup
x = soup.find_all("h2", attrs = {"class": "f18"})
len(x)

In [ ]:
x[1].find("")

In [ ]:
title = x[0].find("a").get("data-content-title")
href = x[0].find("a").get("href")


In [ ]:
import requests
import os
API_KEY = os.environ.get('GOOGLE_NEWS_API_KEY')
TOPIC = "coronavirus"
COUNTRY = "my"

In [ ]:
# x = requests.get(f"https://newsapi.org/v2/top-headlines?q={TOPIC}&country={COUNTRY}&apiKey={API_KEY}")
x2 = requests.get(f"https://newsapi.org/v2/top-headlines?q={TOPIC}&country={COUNTRY}&apiKey={API_KEY}")

In [ ]:
# q = x.json()
# len(q["articles"])
q2 = x2.json()
q2["articles"]

In [ ]:


from newsapi import NewsApiClient

# Init
newsapi = NewsApiClient(api_key='API_KEY')

# /v2/top-headlines
top_headlines = newsapi.get_top_headlines(q='bitcoin',
                                          sources='bbc-news,the-verge',
                                          category='business',
                                          language='en',
                                          country='us')

# /v2/everything
all_articles = newsapi.get_everything(q='bitcoin',
                                      sources='bbc-news,the-verge',
                                      domains='bbc.co.uk,techcrunch.com',
                                      from_param='2017-12-01',
                                      to='2017-12-12',
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)

# /v2/sources
sources = newsapi.get_sources()



In [ ]:
# 'img_src': response.css('div.td-post-content img::attr(src)').getall(),
import pymongo

client = pymongo.MongoClient()
db = client["news"]
coll = db["sebenarnya_v2_test1"]
# coll = db["sebenarnya_v2_proccessed1"]

q = {'category': 'COVID-19'}

In [ ]:
len(list(coll.find()))

In [ ]:
x = [j["date"] for j in list(coll.find())]
x

In [ ]:
coll.find_one()["content_text"]

In [ ]:
# list(coll.find(q))

In [ ]:
x = coll.find_one()
x.keys()

In [ ]:
x["date"]

In [ ]:
q = coll.list_indexes()
list(q)

In [ ]:
coll.drop_indexes()

In [ ]:
coll.create_index([('search_text', TEXT), ('title', TEXT)])

In [ ]:
x = coll.find({"$text":{"$search": "\"Waspada Mesej Penipuan\""}})
# x = coll.find()
z = list(x)
len(z)

In [ ]:
# z[]

In [ ]:
%%time
x = coll.find_one()
list(x.keys())

In [ ]:
x["content_text"]

In [ ]:
list(coll.find_one().keys())



In [ ]:
import datetime
dt = datetime.datetime(2020, 4,1)
category = "placeholder"
enabled_projection = ["url", "title", "category"]
proj = {key:1 for key in enabled_projection}
filter_query = {"$text": {"$search":"Palsu"}, "category": category}
x = list(coll.find(filter_query, proj).limit(2))

In [ ]:
len(x)

In [ ]:
content = coll.find_one()['content_html']
soup = BeautifulSoup(content[0], 'html.parser')
[line.strip() for line in soup.text.split('\n') if line.strip() != '']

In [ ]:
len(list(coll.find()))

In [ ]:
# coll.drop()